In [ ]:
import sys
import os
sys.path.append('./src')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from src.CRAIGSLISTBARGAIN import *
from src.dataset import SimpleTextDataset
from sklearn.metrics import r2_score

In [3]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm.auto import tqdm

from dataset import TextDataset 

import time

# from CaSiNo import *
from common import *

tic, toc = (time.time, time.time)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model.cuda();
model.eval();

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
train = fetch_and_prepare_dataset('./dataset/CRAIGSLISTBARGAIN/train.json')
# valid = fetch_and_prepare_dataset('./dataset/CRAIGSLISTBARGAIN/valid.json')
valid = fetch_and_prepare_dataset('./dataset/CRAIGSLISTBARGAIN/test.json')

In [7]:
train['quintile'] = pd.qcut(train['buyer_price'], q=20, labels=[int(i) for i in range(1, 21)])
train_dialogs = train[(train.chat_log.apply(lambda item:len(item)>8))&(train['quintile']<18)].apply(lambda row: causal_dialogue_merge(row, ), axis=1).to_list()
valid_dialogs = valid[(valid.chat_log.apply(lambda item:len(item)>8))&(valid['buyer_price']<2450)].apply(lambda row: causal_dialogue_merge(row, ), axis=1).to_list()

In [8]:
X_train = [(item['hidden_states'][5], item['hidden_states'][15], item['hidden_states'][25]) for item in SimpleTextDataset(train_dialogs, tokenizer=tokenizer, model=model, residual_stream=True)]
y_train = train[(train.chat_log.apply(lambda item:len(item)>8))&(train['quintile']<18)]['buyer_price'].to_list()

X_valid = [(item['hidden_states'][5], item['hidden_states'][15], item['hidden_states'][25]) for item in SimpleTextDataset(valid_dialogs, tokenizer=tokenizer, model=model, residual_stream=True)]
y_valid = valid[(valid.chat_log.apply(lambda item:len(item)>8))&(valid['buyer_price']<2450)]['buyer_price'].to_list()


  0%|          | 0/2643 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

In [9]:
param_grid = {
            'pca__n_components': [n for n in range(0, 500, 100)],
            'regressor__alpha': [0],
        }
for layer in range(3):
    X = [item[layer].squeeze().numpy() for item in X_train]
    scaler = StandardScaler()
    transformed_y = scaler.fit_transform(np.array(y_train).reshape(-1, 1))
    best_estimator_, best_score_, gs_best_params = fit_ridge_regression_pipeline(X, transformed_y, param_grid=param_grid)
    XT = [item[layer].squeeze().numpy() for item in X_valid]
    pred = best_estimator_.predict(XT)
    scaled_pred = scaler.inverse_transform(pred)
    base_r2 = r2_score(y_valid, scaled_pred)
    print(f'For the layer {[5, 15, 25][layer]} R^2 score for the buyer_price: {round(base_r2, 2)}')


For the layer 5 R^2 score for the buyer_price: 0.41
For the layer 15 R^2 score for the buyer_price: 0.4
For the layer 25 R^2 score for the buyer_price: 0.45


In [5]:
train = fetch_and_prepare_dataset('./dataset/CRAIGSLISTBARGAIN/train.json')
# valid = fetch_and_prepare_dataset('./dataset/CRAIGSLISTBARGAIN/valid.json')
valid = fetch_and_prepare_dataset('./dataset/CRAIGSLISTBARGAIN/test.json')

In [6]:
train['quintile'] = pd.qcut(train['seller_price'], q=20, labels=[int(i) for i in range(1, 21)])
train_dialogs = train[(train.chat_log.apply(lambda item:len(item)>8))&(train['quintile']<18)].apply(lambda row: causal_dialogue_merge(row, ), axis=1).to_list()
valid_dialogs = valid[(valid.chat_log.apply(lambda item:len(item)>8))&(valid['seller_price']<3400)].apply(lambda row: causal_dialogue_merge(row, ), axis=1).to_list()

In [7]:
X_train = [(item['hidden_states'][5], item['hidden_states'][15], item['hidden_states'][25]) for item in SimpleTextDataset(train_dialogs, tokenizer=tokenizer, model=model, residual_stream=True)]
y_train = train[(train.chat_log.apply(lambda item:len(item)>8))&(train['quintile']<18)]['seller_price'].to_list()

X_valid = [(item['hidden_states'][5], item['hidden_states'][15], item['hidden_states'][25]) for item in SimpleTextDataset(valid_dialogs, tokenizer=tokenizer, model=model, residual_stream=True)]
y_valid = valid[(valid.chat_log.apply(lambda item:len(item)>8))&(valid['seller_price']<3400)]['seller_price'].to_list()


  0%|          | 0/2630 [00:00<?, ?it/s]

  0%|          | 0/424 [00:00<?, ?it/s]

In [8]:
param_grid = {
            'pca__n_components': [n for n in range(0, 500, 100)],
            'regressor__alpha': [0],
        }
for layer in range(3):
    X = [item[layer].squeeze().numpy() for item in X_train]
    scaler = StandardScaler()
    transformed_y = scaler.fit_transform(np.array(y_train).reshape(-1, 1))
    best_estimator_, best_score_, gs_best_params = fit_ridge_regression_pipeline(X, transformed_y, param_grid=param_grid)
    XT = [item[layer].squeeze().numpy() for item in X_valid]
    pred = best_estimator_.predict(XT)
    scaled_pred = scaler.inverse_transform(pred)
    base_r2 = r2_score(y_valid, scaled_pred)
    print(f'For the layer {[5, 15, 25][layer]} R^2 score for the Seller_price: {round(base_r2, 2)}')


For the layer 5 R^2 score for the Seller_price: 0.5
For the layer 15 R^2 score for the Seller_price: 0.36
For the layer 25 R^2 score for the Seller_price: 0.46
